# Basic RAG setup
Here we will demonstrate how to run a local RAG setup where both the vetor store and the language models (embedding and LLM) are local. This can be desireable if you want to make internal documents *chattable* but do not wish to sent them across the Internet for processing in the cloud.


# Deployment of models
We deploy the Ollama models using the command `make fetch_models`.

In [1]:
import json
import requests

def query_raw_model(query):
    r = requests.post(
        'http://localhost:11434/api/chat',
        data=json.dumps({
            'model': 'llama3.2',
            "messages": [{
                "role": "user",
                "content": query,
            }],
            "stream": False
        })
    )
    r.raise_for_status()

    return r.json()['message']['content']


We will now ask the model the following question:

> "Under the Napoleonic wars, which countries took part in the fifth coalition against France?"

As LLMs have randomness built in, we repeat the query five time. The core members of the fifth coalition were the Austrian Empire and United Kingdom. From the five answers below, we see that the model either upfront states that it does not know the answers or generates lists that include more countries than the actual members of the coalition.

A larger model such as ChatGPT would likely have answered correctly but that is besides the point. We are considering the case of internal documnets which neither ChatGPT nor out local llama model knows about. We will now move on seting up the vector database containing the internal documents (or our case the wikipedia page on the Napoleonic Wars).

In [2]:
q = "Under the Napoleonic wars, which countries took part in the fifth coalition against France?"

responsees = [
    query_raw_model(q) for i in range(5)
]

In [3]:
for response in responsees:
    print(response)
    print("-" * 100)


The Fifth Coalition was formed against Napoleon Bonaparte's French Empire in 1809. The countries that made up this coalition were:

1. Austria
2. Britain (United Kingdom)
3. Russia
4. Sweden-Norway
5. Ottoman Empire 

These countries joined forces to counter the expansion of Napoleon's power and influence in Europe during his reign.
----------------------------------------------------------------------------------------------------
The Fifth Coalition against France consisted of Austria and Russia. The other main member was Sweden, but it joined later in the year after Sweden broke its alliance with Prussia.

However, some sources say it included Prussia also, although there is no record of them actually declaring war.
----------------------------------------------------------------------------------------------------
The Fifth Coalition was formed against Napoleon Bonaparte's French Empire. The main countries that took part in this alliance were:

1. Russia
2. Austria
3. Sweden-Norway

## Weaviate vector store

The script `populate_db.py` will fetch Napoleonic Wars Wikipedia page. Each section on the page is split into chunks using an instance of the `RecursiveCharacterTextSplitter` from LangChain. This ensures that chunk divisions follows the section layout of the page wich should ensure higher quality of the chunks.
Each chunk is then stored in the Weaviate database including the relevant section title and an embedding vector.

In [4]:
from scripts import populate_db
populate_db.run()

Imported 155 chunks into the Napoleonic Wars collection


In [2]:
import weaviate

client = weaviate.connect_to_local()
collection = client.collections.get('napoleonic_wars')

for item in collection.iterator(include_vector=True):
    print(f'Section title: {item.properties["title"]}')
    print(f'Chunk text: {item.properties["text"][0:100]} ...')
    print(f'Embedding vector: {item.vector["default"][0:5]} ...')
    break

client.close()

Section title: Invasion of Russia, 1812
Chunk text: The central issue for both Emperor Napoleon I and Tsar Alexander I was control over Poland. Each wan ...
Embedding vector: [-0.00635263929143548, 0.03718530014157295, -0.15025673806667328, -0.012791609391570091, 0.0685529112815857] ...


With the vector database set up, we are now ready to build our RAG pipeline.

## RAG

In [1]:
from backend import rag

my_RAGAgent = rag.LocalRAGAgent()

In [2]:
docs = my_RAGAgent.vector_store.similarity_search_with_score("Under the Napoleonic wars, which countries took part in the fifth coalition against France?")

In [3]:
docs[0]

(Document(metadata={'title': 'War of the Fifth Coalition, 1809'}, page_content="The Fifth Coalition (1809) of Britain and Austria against France formed as Britain engaged in the Peninsular War in Spain and Portugal. The sea became a major theatre of war against Napoleon's allies. Austria, previously an ally of France, took the opportunity to attempt to restore its imperial territories in Germany as held prior to Austerlitz. During the time of the Fifth Coalition, the Royal Navy won a succession of victories in the French colonies. On land the major battles included Battles"),
 1.0)

In [4]:
query = "Under the Napoleonic wars, which countries took part in the fifth coalition against France?"

for event in my_RAGAgent.stream(query):
    event["messages"][-1].pretty_print()



================================ Human Message =================================

Under the Napoleonic wars, which countries took part in the fifth coalition against France?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (4b916497-67e8-4558-a7d7-5fd48f697640)
 Call ID: 4b916497-67e8-4558-a7d7-5fd48f697640
  Args:
    query: Fifth Coalition against France


NameError: name 'self' is not defined

================================ Human Message =================================

Under the Napoleonic wars, which countries took part in the fifth coalition against France?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (b067b5c7-eac0-42ed-8508-d869fed1fed3)
 Call ID: b067b5c7-eac0-42ed-8508-d869fed1fed3
  Args:
    query: Fifth Coalition against France
================================= Tool Message =================================
Name: retrieve_context

Source: {'title': 'War of the Fifth Coalition, 1809'}
Content: The Fifth Coalition (1809) of Britain and Austria against France formed as Britain engaged in the Peninsular War in Spain and Portugal. The sea became a major theatre of war against Napoleon's allies. Austria, previously an ally of France, took the opportunity to attempt to restore its imperial territories in Germany as held prior to Austerlitz. During the time of the Fifth Coalition, the Royal Navy won a s